In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/My Drive/Colab Notebooks') # Replace with the actual path



Mounted at /content/drive


In [ ]:
import nbformat

with open('/content/drive/My Drive/Colab Notebooks/DIP_proj/read_data.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute each cell
for cell in notebook.cells:
    if cell.cell_type == 'code':
        exec(cell.source)

with open('/content/drive/My Drive/Colab Notebooks/DIP_proj/custom_model_1.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute each cell
for cell in notebook.cells:
    if cell.cell_type == 'code':
        exec(cell.source)

with open('/content/drive/My Drive/Colab Notebooks/DIP_proj/custom_model_2.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute each cell
for cell in notebook.cells:
    if cell.cell_type == 'code':
        exec(cell.source)

with open('/content/drive/My Drive/Colab Notebooks/DIP_proj/preprocess_data_1.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute each cell
for cell in notebook.cells:
    if cell.cell_type == 'code':
        exec(cell.source)

with open('/content/drive/My Drive/Colab Notebooks/DIP_proj/preprocess_data_2.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute each cell
for cell in notebook.cells:
    if cell.cell_type == 'code':
        exec(cell.source)

with open('/content/drive/My Drive/Colab Notebooks/DIP_proj/custom_model_3.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute each cell
for cell in notebook.cells:
    if cell.cell_type == 'code':
        exec(cell.source)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


In [ ]:
import numpy as np

# first one
# from custom_model_1 import create_model
# from preprocess_data_1 import create_random_federated_data
# from read_data import load_cifar10
num_clients = 5
num_clients = 5
train_data, test_data = load_cifar10()
print("Data loaded")
federated_data = create_random_federated_data(train_data[0], train_data[1], num_clients)


Data loaded


In [ ]:
# second one
# from custom_model_2 import create_model
# from read_data import load_cifar10
# from preprocess_data_2 import split_data_by_category
train_data, test_data = load_cifar10()
split_x_train, split_y_train = split_data_by_category(train_data[0], train_data[1])
split_x_test, split_y_test = split_data_by_category(test_data[0], test_data[1])


In [ ]:
def encode_weights(weights):
    return [w + 1 for w in weights]

def decode_weights(weights):
    return [w - 1 for w in weights]

In [ ]:
def train_on_client(model, client_data, epochs=10):
    x_train, y_train = client_data
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, verbose=0)


In [ ]:
def aggregate_models(models):
    # weights = [model.get_weights() for model in models]
    decrypted_weights = [decode_weights(weights) for weights in models]
    avg_weights = [np.mean(np.array(w), axis=0) for w in zip(*decrypted_weights)]
    return avg_weights

In [ ]:
def train_model(fedarated_data, model_type, test_data):
# Initialize the model
    if model_type == 1:
        global_model = create_model_1()
    if model_type == 2:
        global_model = create_model_2()
    else:
        global_model = create_model_3()
    # global_model = create_model_1()
    global_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    # Federated training process
    num_rounds = 50
    for round_num in range(num_rounds):
        print(f'Round {round_num + 1}')
        encrypted_updates = []

        for client_data in federated_data:
            if model_type == 1:
                client_model = create_model_1()
            elif model_type ==2:
                client_model = create_model_2()
            else:
                client_model = create_model_3()

            client_model.set_weights(global_model.get_weights())
            train_on_client(client_model, client_data, epochs=1)
            # local_models.append(client_model)

            # Encrypt the weights
            encrypted_weights = encode_weights(client_model.get_weights())
            encrypted_updates.append(encrypted_weights)

        # Aggregate models
        global_weights = aggregate_models(encrypted_updates)
        global_model.set_weights(global_weights)

    # Evaluate the global model
    global_model.evaluate(test_data[0], test_data[1])
    return global_model


In [ ]:
model_1_rf = train_model(federated_data, 1, test_data)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7277 - loss: 1.1632


In [ ]:
pred= model_1_rf.predict(test_data[0])
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_data[1], np.argmax(pred, axis=1)))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
              precision    recall  f1-score   support

           0       0.75      0.76      0.76      1000
           1       0.87      0.80      0.83      1000
           2       0.60      0.66      0.63      1000
           3       0.57      0.54      0.55      1000
           4       0.71      0.68      0.69      1000
           5       0.66      0.62      0.64      1000
           6       0.76      0.82      0.79      1000
           7       0.76      0.74      0.75      1000
           8       0.81      0.85      0.83      1000
           9       0.79      0.81      0.80      1000

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000



In [ ]:
model_1_rf.save('/content/drive/My Drive/Colab Notebooks/DIP_proj/model_1_rf.h5')

In [ ]:
model_1_cd = train_model([split_x_train, split_y_train], 1, test_data)

Round 1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7285 - loss: 1.1288


In [ ]:
pred= model_1_cd.predict(test_data[0])
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_data[1], np.argmax(pred, axis=1)))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
              precision    recall  f1-score   support

           0       0.76      0.76      0.76      1000
           1       0.82      0.88      0.85      1000
           2       0.63      0.61      0.62      1000
           3       0.57      0.56      0.56      1000
           4       0.70      0.69      0.69      1000
           5       0.63      0.66      0.65      1000
           6       0.82      0.75      0.78      1000
           7       0.72      0.81      0.76      1000
           8       0.81      0.82      0.82      1000
           9       0.83      0.76      0.79      1000

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000



In [ ]:
model_1_cd.save('/content/drive/My Drive/Colab Notebooks/DIP_proj/model_1_cd.h5')

In [ ]:
model_2_rf = train_model(federated_data, 2, test_data)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8074 - loss: 0.5541


In [ ]:
pred= model_2_rf.predict(test_data[0])
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_data[1], np.argmax(pred, axis=1)))

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
              precision    recall  f1-score   support

           0       0.82      0.83      0.83      1000
           1       0.93      0.90      0.91      1000
           2       0.78      0.68      0.72      1000
           3       0.74      0.51      0.60      1000
           4       0.70      0.84      0.76      1000
           5       0.74      0.71      0.72      1000
           6       0.74      0.91      0.82      1000
           7       0.85      0.86      0.85      1000
           8       0.86      0.92      0.89      1000
           9       0.88      0.89      0.89      1000

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



In [ ]:
model_2_rf.save('/content/drive/My Drive/Colab Notebooks/DIP_proj/model_2_rf.h5')

In [ ]:
model_2_cd = train_model([split_x_train, split_y_train], 2, test_data)

Round 1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8148 - loss: 0.5445


In [ ]:
pred= model_2_cd.predict(test_data[0])
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_data[1], np.argmax(pred, axis=1)))

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
              precision    recall  f1-score   support

           0       0.85      0.82      0.83      1000
           1       0.94      0.89      0.91      1000
           2       0.80      0.67      0.73      1000
           3       0.71      0.60      0.65      1000
           4       0.70      0.84      0.77      1000
           5       0.75      0.72      0.73      1000
           6       0.79      0.90      0.84      1000
           7       0.87      0.84      0.85      1000
           8       0.86      0.93      0.89      1000
           9       0.86      0.92      0.89      1000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



In [ ]:
model_2_cd.save('/content/drive/My Drive/Colab Notebooks/DIP_proj/model_2_cd.h5')


In [ ]:
model_3_rf = train_model(federated_data, 3, test_data)


Round 1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7498 - loss: 0.8260


In [ ]:
pred= model_3_rf.predict(test_data[0])
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_data[1], np.argmax(pred, axis=1)))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
              precision    recall  f1-score   support

           0       0.82      0.75      0.78      1000
           1       0.88      0.85      0.87      1000
           2       0.69      0.60      0.64      1000
           3       0.57      0.55      0.56      1000
           4       0.72      0.67      0.69      1000
           5       0.61      0.71      0.66      1000
           6       0.82      0.80      0.81      1000
           7       0.74      0.80      0.77      1000
           8       0.81      0.88      0.84      1000
           9       0.80      0.83      0.81      1000

    accuracy                           0.74     10000
   macro avg       0.75      0.74      0.74     10000
weighted avg       0.75      0.74      0.74     10000



In [ ]:
model_3_rf.save('/content/drive/My Drive/Colab Notebooks/DIP_proj/model_3_rf.h5')


In [ ]:
model_3_cd = train_model([split_x_train, split_y_train], 3, test_data)


Round 1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7395 - loss: 0.8555


In [ ]:
pred= model_3_cd.predict(test_data[0])
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_data[1], np.argmax(pred, axis=1)))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
              precision    recall  f1-score   support

           0       0.77      0.78      0.78      1000
           1       0.85      0.85      0.85      1000
           2       0.66      0.61      0.63      1000
           3       0.55      0.55      0.55      1000
           4       0.69      0.66      0.67      1000
           5       0.63      0.69      0.66      1000
           6       0.83      0.76      0.79      1000
           7       0.74      0.78      0.76      1000
           8       0.83      0.85      0.84      1000
           9       0.80      0.81      0.81      1000

    accuracy                           0.73     10000
   macro avg       0.74      0.73      0.73     10000
weighted avg       0.74      0.73      0.73     10000



In [ ]:
model_3_cd.save('/content/drive/My Drive/Colab Notebooks/DIP_proj/model_3_cd.h5')
